In [464]:
import pandas as pd
import glob
import os
import json
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import psycopg2

In [465]:
# Define your database connection parameters
with open("connection_info.json", "r") as file:
    conn_info = json.load(file)
    print("done reading file")

done reading file


In [466]:
conn_string = f"postgresql://{conn_info['db_user']}:{conn_info['db_password']}@{conn_info['db_host']}:{conn_info['db_port']}/{conn_info['db_name']}"
# # Create a database engine with connection pooling
engine = create_engine(conn_string, pool_pre_ping=True)

session = sessionmaker(bind=engine)

conn = psycopg2.connect(conn_string)
print(conn)

<connection object at 0x7fbc9332ccc0; dsn: 'user=casey password=xxx dbname=airbnb host=localhost port=5432', closed: 0>


In [467]:
# A session has been created now let's do some basic queries.
with conn.cursor() as cursor:
    cursor.execute("SELECT * FROM reviews")
    rows = cursor.fetchall()
    cursor.execute("SELECT * FROM listings")

In [468]:
sql1 = """SELECT *,
   ABS(CAST(REPLACE(REPLACE(Price,'$',''), ',', '') AS NUMERIC)) AS price_clean,
   CASE WHEN instant_bookable = 't' THEN TRUE ELSE FALSE END AS instant_bookable_boolean
   from listings;"""
cursor2 = conn.cursor()
cursor2.execute(sql1)
# Now update the entries in the database


rows = cursor2.fetchall()
cols = [desc[0] for desc in cursor2.description]
df = pd.DataFrame(rows, columns=cols)
df

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,price_clean,instant_bookable_boolean,price_clean,instant_bookable_boolean
0,1223704,https://www.airbnb.com/rooms/1223704,20231204023824,2023-12-04,city scrape,Townhouse in San Francisco · ★4.79 · 2 bedroom...,None,Welcome to the heart of San Francisco's dynami...,https://a0.muscache.com/pictures/miso/Hosting-...,1588519,...,f,1,1,0,0,0.24,187.00,False,187.00,False
1,33495561,https://www.airbnb.com/rooms/33495561,20231204023824,2023-12-04,city scrape,Guest suite in San Francisco · ★4.82 · 2 bedro...,None,"Located in the north of Golden Gate Park, this...",https://a0.muscache.com/pictures/bc151c72-0721...,13630570,...,f,1,1,0,0,6.42,189.00,False,189.00,False
2,1036860,https://www.airbnb.com/rooms/1036860,20231204023824,2023-12-04,city scrape,Rental unit in San Francisco · ★4.95 · 2 bedro...,None,Near Hyde pier and cable car station,https://a0.muscache.com/pictures/29216510/8f23...,5708168,...,f,1,1,0,0,0.16,270.00,False,270.00,False
3,38263835,https://www.airbnb.com/rooms/38263835,20231204023824,2023-12-04,previous scrape,Rental unit in San Francisco · 1 bedroom · 1 b...,None,None,https://a0.muscache.com/pictures/c5dabdb0-aa7c...,1144980,...,f,2,0,2,0,NaN,90.00,False,90.00,False
4,37686008,https://www.airbnb.com/rooms/37686008,20231204023824,2023-12-04,previous scrape,Boutique hotel in San Francisco · 1 bedroom · ...,None,So many top attractions are within walking dis...,https://a0.muscache.com/pictures/3a74e27b-106c...,47514427,...,f,23,1,22,0,NaN,366.00,False,366.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,1014574177559636710,https://www.airbnb.com/rooms/1014574177559636710,20231204023824,2023-12-04,city scrape,Rental unit in San Francisco · ★New · 1 bedroo...,None,None,https://a0.muscache.com/pictures/miso/Hosting-...,436782538,...,t,1,1,0,0,1.00,106.00,True,106.00,True
8052,811399316397850759,https://www.airbnb.com/rooms/811399316397850759,20231204023824,2023-12-04,city scrape,Condo in San Francisco · Studio · 1 bed · 1 bath,None,None,https://a0.muscache.com/pictures/prohost-api/H...,410094341,...,t,37,37,0,0,NaN,261.00,True,261.00,True
8053,934838802204780322,https://www.airbnb.com/rooms/934838802204780322,20231204023824,2023-12-04,city scrape,Home in San Francisco · 1 bedroom · 1 bed · 1 ...,None,None,https://a0.muscache.com/pictures/74abb8db-8630...,510857674,...,t,2,0,2,0,NaN,62.00,True,62.00,True
8054,1003986068740979231,https://www.airbnb.com/rooms/1003986068740979231,20231204023824,2023-12-04,city scrape,Home in San Francisco · ★New · 1 bedroom · 1 b...,None,None,https://a0.muscache.com/pictures/56bcb769-2658...,188629986,...,f,4,0,3,1,NaN,56.00,False,56.00,False


In [469]:
print(df.columns)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [470]:
alter_sql = """
ALTER TABLE listings
    ADD COLUMN IF NOT EXISTS price_clean NUMERIC,
    ADD COLUMN IF NOT EXISTS instant_bookable_boolean BOOLEAN;
"""
# Execute the ALTER TABLE statement
cursor = conn.cursor()
cursor.execute(alter_sql)

# Populate the new columns with data
update_sql = """
UPDATE listings
SET price_clean = ABS(CAST(REPLACE(REPLACE(Price,'$',''), ',', '') AS NUMERIC)),
    instant_bookable_boolean = (CASE WHEN instant_bookable = 't' THEN TRUE ELSE FALSE END);
"""

# Execute the UPDATE statement
cursor.execute(update_sql)

# Commit the changes
conn.commit()

In [471]:
query = """SELECT price_clean, instant_bookable_boolean, neighbourhood_cleansed FROM listings
"""
cursor.execute(query)
rows = cursor.fetchall()
cols = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=cols)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8056 entries, 0 to 8055
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   price_clean               8056 non-null   object
 1   instant_bookable_boolean  8056 non-null   bool  
 2   neighbourhood_cleansed    8056 non-null   object
dtypes: bool(1), object(2)
memory usage: 133.9+ KB


In [472]:
df.describe()

,price_clean,instant_bookable_boolean,neighbourhood_cleansed
count,8056,8056,8056
unique,675,2,36
top,99.00,False,Downtown/Civic Center
freq,198,5911,1010


In [473]:
df.neighbourhood_cleansed

0                  Mission
1       Financial District
2                Glen Park
3               Twin Peaks
4         Western Addition
               ...        
8051      Western Addition
8052              Nob Hill
8053     Visitacion Valley
8054       South of Market
8055       South of Market
Name: neighbourhood_cleansed, Length: 8056, dtype: object

Question 1: What is the expected revenue for hosts?
In order to do this we need to calculate price_clean * (30 - booked_out_30) for every row in the data set and then group by host_id. This will give us the total expected revenue for the host over the next 30 days.

In [474]:
host_revenue_query = """
SELECT 
    host_id,
    host_name,
    SUM(price_clean * (30 - availability_30)) AS expected_revenue
FROM 
    listings
GROUP BY
    host_id, host_name
ORDER BY
    expected_revenue DESC;
"""


cursor.execute(host_revenue_query)
rows = cursor.fetchall()
cols = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=cols)
df

,host_id,host_name,expected_revenue
0,91333416,Jan,16521184.00
1,173206762,Stratford,3096238.00
2,283380571,Galleria Park,1799820.00
3,268186698,Executive Hotel Vintage Court,1199880.00
4,501999278,RoomPicks,986485.00
...,...,...,...
3963,79711096,Patricia,0.00
3964,48502153,Sarah,0.00
3965,495796,Joy,0.00
3966,172548751,Alex,0.00


I repeat the same query but this time for the expected revenue over the next 60 days.

In [475]:
host_revenue_query2 = """
SELECT 
    host_id,
    host_name,
    SUM(price_clean * (60 - availability_60)) AS expected_revenue
FROM 
    listings
GROUP BY
    host_id, host_name
ORDER BY
    expected_revenue DESC;
"""


cursor.execute(host_revenue_query2)
rows = cursor.fetchall()
cols = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=cols)
df

,host_id,host_name,expected_revenue
0,91333416,Jan,33033124.00
1,173206762,Stratford,4126162.00
2,283380571,Galleria Park,3599640.00
3,268186698,Executive Hotel Vintage Court,2399760.00
4,107434423,Blueground,1641397.00
...,...,...,...
3963,192964563,Benjamin,0.00
3964,825393,Catherine,0.00
3965,225081901,Angela,0.00
3966,44354824,Laura,0.00


Question 2: Which hosts have the highest ratings on their listings?
First I average the review score rating for each host and then I filter out the hosts who have less than 3 reviews and no reviews in the last month.

In [477]:
host_rating_query = """
SELECT 
    host_id,
    host_name,
    sum_number_of_reviews,
    sum_number_of_reviews_ltm,
    average_rating
FROM (
    SELECT 
        host_id,
        host_name,
        SUM(number_of_reviews) AS sum_number_of_reviews,
        SUM(number_of_reviews_ltm) AS sum_number_of_reviews_ltm,
        AVG(review_scores_rating) AS average_rating
    FROM 
        listings
    GROUP BY
        host_id, host_name
) AS subquery
WHERE
    sum_number_of_reviews > 3 
    AND sum_number_of_reviews_ltm > 0
ORDER BY
    average_rating DESC;
"""


cursor.execute(host_rating_query)
rows = cursor.fetchall()
cols = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=cols)
df

,host_id,host_name,sum_number_of_reviews,sum_number_of_reviews_ltm,average_rating
0,58041824,Annika,15,3,5.000000
1,535845,Joanne,5,1,5.000000
2,401165959,Keyur,4,1,5.000000
3,36084845,Seva,7,7,5.000000
4,11968934,Shayan,5,3,5.000000
...,...,...,...,...,...
2616,12491207,Choon Hee,5,5,3.200000
2617,542041520,Allen,35,35,3.024815
2618,477980041,Sam,34,22,2.910000
2619,520931919,Chris,22,22,2.860000


Question 3: Which neighborhoods are consistently the most booked?
First I average the availability over the next 365 days of the listings in each neighborhood and then filter out the neighborhoods with less than 5 listings.

In [478]:
neighborhood_query = """
SELECT
    neighborhood,
    avg_availability,
    num_listings
FROM (
    SELECT
        neighbourhood_cleansed AS neighborhood,
        AVG(availability_365) AS avg_availability,
        COUNT(DISTINCT id) AS num_listings
    FROM 
        listings
    GROUP BY 
        neighbourhood_cleansed
) AS subquery
WHERE
    num_listings > 5
ORDER BY 
    avg_availability ASC;
"""

cursor.execute(neighborhood_query)
rows = cursor.fetchall()
cols = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=cols)
df

,neighborhood,avg_availability,num_listings
0,Seacliff,113.8888888888888889,9
1,Bernal Heights,139.6566265060240964,332
2,Noe Valley,139.6793893129770992,262
3,Outer Richmond,146.0050761421319797,197
4,Russian Hill,153.1405405405405405,185
5,Financial District,154.6424870466321244,193
6,Mission,157.1618759455370651,661
7,Castro/Upper Market,157.5192307692307692,312
8,Potrero Hill,157.8176470588235294,170
9,Parkside,158.5095541401273885,157


Question 4: What is the average price of each neighborhood?
This is a very similar query to the one above but just averages the price instead of availability.

In [479]:
neighborhood_query = """
SELECT
    neighborhood,
    avg_price,
    num_listings
FROM (
    SELECT
        neighbourhood_cleansed AS neighborhood,
        AVG(price_clean) AS avg_price,
        COUNT(DISTINCT id) AS num_listings
    FROM 
        listings
    GROUP BY 
        neighbourhood_cleansed
) AS subquery
WHERE
    num_listings > 5
ORDER BY 
    avg_price ASC;
"""

cursor.execute(neighborhood_query)
rows = cursor.fetchall()
cols = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=cols)
df

,neighborhood,avg_price,num_listings
0,Crocker Amazon,99.8888888888888889,45
1,Excelsior,113.8830409356725146,171
2,Visitacion Valley,127.4404761904761905,84
3,Bayview,129.0670391061452514,179
4,Ocean View,131.1395348837209302,129
5,Lakeshore,132.4626865671641791,67
6,Parkside,148.0828025477707006,157
7,Outer Sunset,157.4194444444444444,360
8,Outer Mission,163.7840000000000000,125
9,West of Twin Peaks,181.2596153846153846,104
